# Column 처리방법 9 가지

#### 1st categorical_columns_with_identity
#### 2nd categorical_columns_with_vocabulary_file
#### 3rd categorical_columns_with_vocabulary_list
#### 4th categorical_columns_with_hash_bucket
#### 5th crossed_column
#### 6th numberic_column
#### 7th indicator_column
#### 8th embedding_column
#### 9th bucketized_column

In [7]:
from tensorflow import feature_column
from tensorflow.keras import layers

In [8]:
# df = pd.read_csv("/Users/patricklee/Desktop/파이썬/Kaggle/HR/aug_train.csv")
df = pd.read_csv("C:/Users/BIS_COM/data/task/loan_data.csv")

In [9]:
df = df.dropna()

In [10]:
df

,credit.policy,purpose,int.rate,installment,log.annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec,not.fully.paid
0,1,debt_consolidation,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0
1,1,credit_card,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0
2,1,debt_consolidation,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0
3,1,debt_consolidation,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0,0
4,1,credit_card,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9573,0,all_other,0.1461,344.76,12.180755,10.39,672,10474.000000,215372,82.1,2,0,0,1
9574,0,all_other,0.1253,257.70,11.141862,0.21,722,4380.000000,184,1.1,5,0,0,1
9575,0,debt_consolidation,0.1071,97.81,10.596635,13.09,687,3450.041667,10036,82.9,8,0,0,1
9576,0,home_improvement,0.1600,351.58,10.819778,19.18,692,1800.000000,0,3.2,5,0,0,1


In [27]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('not.fully.paid')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [28]:
batch_size = 5 # 예제를 위해 작은 배치 크기를 사용합니다.
df1 = df_to_dataset(df, batch_size=batch_size)

In [29]:
example_batch = next(iter(df1))[0]

In [30]:
def demo(feature_column):
    feature_layer = layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy())

# 피쳐엔지니어링  : 수치화

In [33]:
fico = feature_column.numeric_column("fico")
demo(fico)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[697.]
 [687.]
 [772.]
 [702.]
 [807.]]


# 피쳐엔지니어링 : 버킷화

In [34]:
fico_score = feature_column.bucketized_column(fico, boundaries=[500,600,700,800])
demo(fico_score)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


# 피쳐엔지니어링 : 카테고리화

In [35]:
col_name = df['purpose'].unique()
col_name

array(['debt_consolidation', 'credit_card', 'all_other',
       'home_improvement', 'small_business', 'major_purchase',
       'educational'], dtype=object)

In [36]:
purpose = feature_column.categorical_column_with_vocabulary_list(
      'purpose', col_name)

purpose_one_hot = feature_column.indicator_column(purpose)
demo(purpose_one_hot)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]]


# 피쳐엔지니어링 : 다중카테고리화

In [35]:
col_name = df['purpose'].unique()
col_name

array(['debt_consolidation', 'credit_card', 'all_other',
       'home_improvement', 'small_business', 'major_purchase',
       'educational'], dtype=object)

In [40]:
purpose_mh = feature_column.indicator_column(purpose)
demo(purpose_mh)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]]


# 피쳐엔지니어링 : crossed_column 피처링
### 출처 코드
```python
long_x_lat = tf.feature_column.crossed_column(
    [bucketized_longitude, bucketized_latitude], 5*5)

long_x_lat_ohe = tf.feature_column.indicator_column(long_x_lat)
feature_columns = [long_x_lat_ohe]
```
![one](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile21.uf.tistory.com%2Fimage%2F994C64485BA51CAF1D00B4)



# 피쳐엔지니어링 : embedding_columns 피처링
```python
long_x_lat = tf.feature_column.crossed_column(
    [bucketized_longitude, bucketized_latitude], 5*5)

embed_long_x_lat = tf.feature_column.embedding_column(
    long_x_lat, 5)

feature_columns = [embed_long_x_lat]
```
![two](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile6.uf.tistory.com%2Fimage%2F99D92C475BA51CC71AA34B)

In [65]:
df['purpose'] = df['purpose'].map({'debt_consolidation':0,
                                             'all_other':1,'credit_card':2,
                                             'home_improvement' : 3,
                                             'small_business' : 4,
                                             'major_purchase' : 5,
                                             'educational' : 6,
                                            })

# 예제 회귀 적용

In [74]:
featcols = [
    feature_column.categorical_column_with_vocabulary_list('purpose', col_name),
    feature_column.bucketized_column(fico, boundaries=[500,600,700,800]),
]

In [75]:
def train_input_fn() :
    features = {"purpose":['credit_card','all_other','all_other','credit_card',
                           'all_other','major_purchase','educational','small_business'
                           ,'home_improvement','credit_card'],
               'fico' : [658, 742, 846, 550, 785, 800, 711, 569, 685, 800],
               }
    log_annual_income = [11.350407, 12.15561, 15.564561, 11.784125, 13.45644,
                                10.156456, 11.554651, 13.145645, 11.15616, 10.15651]
    return features, log_annual_income

In [76]:
model = tf.estimator.LinearRegressor(featcols)
model.train(train_input_fn, steps=200)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Public\\Documents\\ESTsoft\\CreatorTemp\\tmp9x5wfngu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_f

In [78]:
def predict_input_fn() :
    features = {
        "purpose" : ['home_improvement','all_other'],
        'fico' : [712, 658]
    }
    return features

predictions = model.predict(predict_input_fn)

print(next(predictions))
print(next(predictions))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Public\Documents\ESTsoft\CreatorTemp\tmp9x5wfngu\model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([10.539863], dtype=float32)}
{'predictions': array([10.873244], dtype=float32)}


# Logistic

In [81]:
# df = pd.read_csv("/Users/patricklee/Desktop/파이썬/Kaggle/HR/aug_train.csv")
df = pd.read_csv("C:/Users/BIS_COM/data/task/loan_data.csv")

In [82]:
from sklearn.model_selection import train_test_split
 
# x(독립변수) y(종속 변수) 분리
x = df.loc[:,'credit.policy':'pub.rec']
y = df['not.fully.paid']
 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)
check = pd.DataFrame(data=df)
check.head()

,credit.policy,purpose,int.rate,installment,log.annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec,not.fully.paid
0,1,debt_consolidation,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0
1,1,credit_card,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0
2,1,debt_consolidation,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0
3,1,debt_consolidation,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0,0
4,1,credit_card,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0,0


In [112]:
x_train = np.array(x_train)
y_train = np.array(y_train)

In [92]:
features = ['credit.policy', 'int.rate', 'installment', 'log.annual.inc',
       'dti', 'fico', 'days.with.cr.line', 'revol.bal', 'revol.util',
       'inq.last.6mths', 'delinq.2yrs', 'pub.rec']
features

['credit.policy',
 'int.rate',
 'installment',
 'log.annual.inc',
 'dti',
 'fico',
 'days.with.cr.line',
 'revol.bal',
 'revol.util',
 'inq.last.6mths',
 'delinq.2yrs',
 'pub.rec']

In [91]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9578 entries, 0 to 9577
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   credit.policy      9578 non-null   int64  
 1   purpose            9578 non-null   object 
 2   int.rate           9578 non-null   float64
 3   installment        9578 non-null   float64
 4   log.annual.inc     9578 non-null   float64
 5   dti                9578 non-null   float64
 6   fico               9578 non-null   int64  
 7   days.with.cr.line  9578 non-null   float64
 8   revol.bal          9578 non-null   int64  
 9   revol.util         9578 non-null   float64
 10  inq.last.6mths     9578 non-null   int64  
 11  delinq.2yrs        9578 non-null   int64  
 12  pub.rec            9578 non-null   int64  
 13  not.fully.paid     9578 non-null   int64  
dtypes: float64(6), int64(7), object(1)
memory usage: 1.0+ MB


In [96]:
for i in features :
    feature_columns = [(feature_column.numeric_column(i))]

In [97]:
classifier = tf.estimator.LinearClassifier(feature_columns=feature_columns,
                                          n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Public\\Documents\\ESTsoft\\CreatorTemp\\tmpsqziiocz', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [113]:
train_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(x=x_train,
                                                             y=y_train,
                                                             batch_size=100,
                                                             num_epochs=None,
                                                             shuffle=True)

In [114]:
classifier.train(input_fn=train_input_fn, steps=10)

INFO:tensorflow:Calling model_fn.


ValueError: features should be a dictionary of `Tensor`s. Given type: <class 'tensorflow.python.framework.ops.Tensor'>